# Pattern 1

## Prepare Resource IDs

まずは作業中に利用する、ECR repository の ID を変数に代入しておく。CDK でデプロイした CloudFormation stack を確認して、以下を編集して代入しておく。

In [ ]:
repository = '...'

また AWS account ID と region ID の情報も必要となるので、こちらも変数に代入しておく。

In [ ]:
import boto3
sts = boto3.client('sts')

account = sts.get_caller_identity()['Account']
account

In [ ]:
region = boto3.Session().region_name
region

## Prepare Model Artifact

モデルのアーティファクト（モデルの重みが保存されているファイル）を準備する。
このパターンでは、 [Amazon SageMaker Examples](https://github.com/aws/amazon-sagemaker-examples) の [PyTorch の MNIST サンプルコード](https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-python-sdk/pytorch_mnist) を利用する。

この notebook を CDK でデプロイする SageMaker notebook instance 上で実行している場合、作業ディレクトリのルートに Amazon SageMaker Examples の Git リポジトリが自動的にクローンされている。もし異なる環境で実行している場合は、GitHub からリポジトリをクローンする。

PyTorch の MNIST のサンプルコードは、一部編集するため、手元にコピーをおいておく。

In [ ]:
!cp -r ../amazon-sagemaker-examples/sagemaker-python-sdk/pytorch_mnist/ .

コピーしたディレクトリにある `pytorch_mnist.ipynb` を実行していく。この notebook を `conda_pytorch_py310` のカーネル（もしくはより高いバージョンの Python のカーネル）で開く。

上のセルから順番に実行していくが、 **最後までは実行せず** に **Host というセクションの直前** まで実行する。ここまで実行すれば、学習が完了し S3 にモデルのアーティファクトがアップロードされる。Host セクションの直前は、9 つ目のコードセルで `estimator.fit()` を実行しているはずである。

後半のセクションで S3 にアップロードされたモデルのパスが必要となるため、最後に実行したコードセルの直後にセルを追加し、 `estimator.model_data` を実行する。実行すると次のような文字列が出力されるはずである。

```
's3://sagemaker-{region}-{account}/pytorch-training-YYYY-MM-DD-HH-mm-ss-fff/output/model.tar.gz'
```

この文字列は後ほど利用するため、変数に代入しておく。

In [ ]:
model_data = 's3://...'

## Prepare Container Image

コンテナイメージを準備する。
このパターンでは、SageMaker が提供するイメージを拡張するため、SageMaker Python SDK の PyTorchModel object を作成し、それが利用するコンテナイメージを取得する。

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(
   entry_point='entry_point.py',
   model_data='s3://DOC-EXAMPLE-BUCKET',
   role=sagemaker.get_execution_role(),
   framework_version='1.11',
   py_version='py38',
)

image_uri = model.serving_image_uri(
   region_name='ap-northeast-1',
   instance_type='ml.m5.large',
)
image_uri

PyTorchModel のコンストラクタに渡す `entry_point` と `model_data` は実際に存在しないファイルとパスでも問題ない。また `framework_version` と `py_version` は、今回モデル学習時の version と一致させるようにしている。

コンテナイメージの URI を取得できたので、これをベースイメージとして拡張する。

In [ ]:
dockerfile = f"""\
FROM {image_uri}
RUN pip install -U sagemaker\
"""

%store dockerfile >Dockerfile

この Dockerfile は SageMaker Python SDK をアップデートするだけのほぼ何も意味を成さないものであるが、例としては問題ない。これをビルドするが、ベースイメージをダウンロードするために、Docker CLI に認証情報を登録しなければならない。認証情報は、自身の AWS account の ECR ではなく、SageMaker 用の PyTorch などのコンテナイメージが保存されている AWS account の ECR registry から取得する。

In [ ]:
registry = image_uri.split('/')[0]
registry

In [ ]:
!aws ecr get-login-password | docker login --username AWS --password-stdin {registry}

そしてコンテナイメージをビルドする。

In [ ]:
!docker image build . -t byoc-inference-pattern1

In [ ]:
!docker image ls

今ビルドしたコンテナイメージを、今度は自身の AWS account の ECR registry の repostiry へプッシュする。そのため、認証情報を再登録する。

In [ ]:
!aws ecr get-login-password | docker login --username AWS --password-stdin {account}.dkr.ecr.{region}.amazonaws.com

そして CDK で作成した ECR repository へプッシュする。

In [ ]:
!docker image tag byoc-inference-pattern1 {account}.dkr.ecr.{region}.amazonaws.com/{repository}:pattern1

In [ ]:
!docker image ls

In [ ]:
!docker image push {account}.dkr.ecr.{region}.amazonaws.com/{repository}:pattern1

## Deploy SageMaker Endpoint

モデルアーティファクトと、コンテナイメージの準備は整ったので、これらを用いて SageMaker endpoint をデプロイする。デプロイするには SageMaker Python SDK の PyTorchModel object を利用する。推論コードは [PyTorch の MNIST サンプルコード](https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-python-sdk/pytorch_mnist) のものをそのまま流用する。

In [ ]:
model = PyTorchModel(
    model_data=model_data,
    image_uri=f'{account}.dkr.ecr.{region}.amazonaws.com/{repository}:pattern1',
    entry_point='./pytorch_mnist/mnist.py',
    role=sagemaker.get_execution_role(),
)

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

## Invoke SageMaker Endpoint

SageMaker endpoint は完了したので、推論処理を実行する。推論処理にはデータが必要なため、また[PyTorch の MNIST サンプルコード](https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-python-sdk/pytorch_mnist) からその部分のコードを拝借する。

In [ ]:
import gzip
import numpy as np
import random
import os

data_dir = "./pytorch_mnist/data/MNIST/raw"
with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "rb") as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)

mask = random.sample(range(len(images)), 16)  # randomly select some of the test images
mask = np.array(mask, dtype=np.int32)
data = images[mask]

In [ ]:
response = predictor.predict(np.expand_dims(data, axis=1))
print("Raw prediction result:")
print(response)
print()

labeled_predictions = list(zip(range(10), response[0]))
print("Labeled predictions: ")
print(labeled_predictions)
print()

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most likely answer: {}".format(labeled_predictions[0]))

以上でパターン 1 は完了である。

## Clean Up

In [ ]:
predictor.delete_endpoint()